# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Открываем и изучаем файл

Импортируем необходимые в работе библиотеки и модули

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
# откроем датасет и сохраним его в переменной df
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
# посмотрим на форму датасета
print(df.shape)

(3214, 5)


In [4]:
# выведем первые пять строк таблицы
display(df.head())

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [5]:
# выведем общую информацию по таблице
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


**Описание данных:**

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

•	`сalls` — количество звонков

•	`minutes` — суммарная длительность звонков в минутах

•	`messages` — количество sms-сообщений

•	`mb_used ` — израсходованный интернет-трафик в Мб

•	`is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0)

Из задания известно (да и видно по нашим выведенным таблицам), что предобработка данных не понадобится так как мы её уже сделали в одном из  предыдущих проектов.

Разделим данные на признаки и целевое значение:

In [6]:
features = df.drop('is_ultra', axis=1) # признаки
target = df['is_ultra'] # целевое значение

In [7]:
# посмотрим на размеры новых данных
print(features.shape, target.shape)

(3214, 4) (3214,)


Как видно данные успешно разделены.

## Разделение исходных данных на выборки

Как сказано в задании разобьем данные на три части: обучающую, валидационную и тестовую. Размеры тестового и валидационного наборов сделаем равными. Исходные данные разобьем в соотношении 3:1:1:
* 1)	Обучающая выборка (train) = 60%
* 2)	Валидационная выборка (valid) = 20%
* 3)	Тестовая выборка (test) = 20%

In [8]:
# разделим данные на обучающую, валидационную и тестовую выборки через numpy.split ()
df_train, df_valid, df_test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])

Проверим как разделились выборки:

In [9]:
print('Доля обучающей выборки равна: {:.0%}'.format(len(df_train) / len(df)))
print('Доля валидационной выборки равна: {:.0%}'.format(len(df_valid) / len(df)))
print('Доля тестовой выборки равна: {:.0%}'.format(len(df_test) / len(df)))

Доля обучающей выборки равна: 60%
Доля валидационной выборки равна: 20%
Доля тестовой выборки равна: 20%


Cоздадим переменные для признаков и целевого признака:

In [10]:
# обучающая выборка
print('обучающая выборка:')
features_train = df_train.drop('is_ultra', axis=1)
target_train = df_train['is_ultra']
print(features_train.shape, target_train.shape)
print()

# валидационная выборка
print('валидационная выборка:')
features_valid = df_valid.drop('is_ultra', axis=1)
target_valid = df_valid['is_ultra']
print(features_valid.shape, target_valid.shape)
print()

# тестовая выборка
print('тестовая выборка:')
features_test = df_test.drop('is_ultra', axis=1)
target_test = df_test['is_ultra']
print(features_test.shape, target_test.shape)

обучающая выборка:
(1928, 4) (1928,)

валидационная выборка:
(643, 4) (643,)

тестовая выборка:
(643, 4) (643,)


Как видим данные разбились на требуемые нам выборки.

## Исследование моделей

### Дерево решений

Первой моделью выберем **`«Дерево решений»`** и, чтобы ее улучшить будем перебирать гиперпараметр `«maх_depth»` в цикле автоматически, не дожидаясь проверки на тестовой выборке:

In [11]:
best_model_tree = None
best_result_tree = 0
best_tree = 0
# выполним цикл для "max_depth" от 1 до 15
for tree in range(1,16):
# обучаем модель с заданной глубиной дерева «tree»    
    model_tree = DecisionTreeClassifier(random_state=13, max_depth=tree) 
    # обучаем модель
    model_tree.fit(features_train, target_train)
    # получаем предсказания модели
    predictions_tree = model_tree.predict(features_valid)
    # посчитаем качество модели    
    result_tree = accuracy_score(target_valid, predictions_tree)
    
    if result_tree > best_result_tree:
        # сохраним наилучшую модель
        best_model_tree = model_tree
        # сохраним наилучшее значение метрики accuracy на валидационных данных
        best_result_tree = result_tree
        # сохраним наилучшее значение гиперпараметра "maх_depth"
        best_tree = tree

print('--------------- Модель "Дерево решений" ---------------')
print()
print("Параметры наилучшей модели на валидационной выборке:")
print("Accuracy =", best_result_tree)
print('Гиперпараметр "maх_depth" =', best_tree)

--------------- Модель "Дерево решений" ---------------

Параметры наилучшей модели на валидационной выборке:
Accuracy = 0.8133748055987559
Гиперпараметр "maх_depth" = 3


### Случайный лес

Второй моделью будет **`«Случайный лес»`**. Настраивать будем количесво деревьев `«n_estimators»`. Чем больше деревьев, тем дольше модель будет учиться, но результат станет лучше (и наоборот). Попробуем на количестве деревьев равное 100 с шагом 10.

In [12]:
best_model_forest = None
best_result_forest = 0
best_forest = 0
for forest in range(1, 101, 10):
    # обучим модель с заданным количеством деревьев
    model_forest = RandomForestClassifier(random_state=13, n_estimators=forest)
    # обучим модель на тренировочной выборке 
    model_forest.fit(features_train, target_train)
    # посчитаем качество модели на валидационной выборке
    result_forest = model_forest.score(features_valid, target_valid)
    if result_forest > best_result_forest:
        # сохраним наилучшую модель
        best_model_forest = model_forest
        # сохраним наилучшее значение метрики accuracy на валидационных данных
        best_result_forest = result_forest
        # сохраним наилучшее значение «n_estimators»
        best_forest = forest


print("--------------- Модель «Случайный лес» ---------------")
print()
print("Параметры наилучшей модели на валидационной выборке:")
print("Accuracy =", best_result_forest)
print("Гиперпараметр «n_estimators» =", best_forest)

--------------- Модель «Случайный лес» ---------------

Параметры наилучшей модели на валидационной выборке:
Accuracy = 0.8180404354587869
Гиперпараметр «n_estimators» = 91


### Логистическая регрессия

Попробуем третий алгоритм — **`«Логистическую регрессию»`**. Обучим модель логистической регрессии на обучающей выборке и посчитаем её качество на валидационных данных.

In [13]:
# Инициализируем модель логистической регрессии с параметром random_state=37
model_logist = LogisticRegression(random_state=13, solver='lbfgs') 

# обучим модель на тренировочной выборке
model_logist.fit(features_train, target_train) 

# получим метрику качества модели на валидационной выборке
result_logist = model_logist.score(features_valid, target_valid) 

print("Accuracy модели логистической регрессии на валидационной выборке:", result_logist)

Accuracy модели логистической регрессии на валидационной выборке: 0.6967340590979783


### Выводы исследования

Лучшей моделью показал себя **`«Случайный лес»`** он самый затратный по ресурсам по сравнению с другими, но в данном проекте требований к данному нюансу не было, поэтому выберем его лучшей моделью.

## Проверка качества лучшей модели на тестовой выборке

Проверим выбранную нами модель **`«Случайного леса»`** на тестовой выборке:

In [14]:
predictions_test = model_forest.predict(features_test)
accuracy_test = accuracy_score(target_test, predictions_test)
print("Качество тестовой выборки:", accuracy_test)

Качество тестовой выборки: 0.7853810264385692


## (бонус) Проверка модели на вменяемость

В качестве параметра вменяемости модели примем, что если наша модель предсказывает лучше чем предположение о том, что количество пользователей использующих тариф «смарт» и «ультра», будет с такими же пропорциями что и в исходной выборке, то она вменяема.
Вычислим количество пользователей использующих тариф «смарт» к общему количеству объектов в выборке:

In [15]:
# проверим какое количество пользователей используют тариф «смарт» и «ультра»
print(df['is_ultra'].value_counts())

0    2229
1     985
Name: is_ultra, dtype: int64


In [16]:
print('Доля пользователей использующих тариф «смарт» равна: {:.0%}'.format((df['is_ultra'] == 0).sum() / df.shape[0]))

Доля пользователей использующих тариф «смарт» равна: 69%


## Вывод

Видим, что доля правильных ответов наших моделей оказалась выше чем у нашего предположения, а следовательно можно считать модели адекватными.